# Getting Radius of Curvature from Images

In [89]:
import os
import datetime

import numpy as np
import pandas
import h5py
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import seaborn as sns
from scipy.optimize import curve_fit
from scipy import integrate
from scipy import optimize

import cv2
import imutils
from imutils import perspective
from imutils import contours

sns.set_context('notebook',font_scale=1.5)
sns.set_style(style='whitegrid')

%matplotlib inline

In [73]:
def make_plot(img,edges,x_center,y_center,radii,xtext,ytext,scale):
    ### Plotting ###
    fig = plt.figure(figsize=(7,20))
    
    ax = fig.add_subplot(311)
    ax.imshow(img,cmap = 'gray')
    ax.set_title('Original Image')
    ax.set_xticks([])
    ax.set_yticks([])
    ax = fig.add_subplot(312)
    ax.imshow(edges,cmap = 'viridis')
    ax.set_title('Edge Image')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # makes copy of the original image
    orig = img.copy()
    ax = fig.add_subplot(313)
    ax.imshow(orig,cmap = 'viridis')
    ax.set_title('Fitted Circles'), 
    ax.set_xticks([])
    ax.set_yticks([])
    for x,y,r,xt,yt in zip(x_center,y_center,radii,xtext,ytext):
        circ = Circle((x,y),r,
                      facecolor='none',
                      edgecolor=(255/256,0,0),
                      fill=False,
                      lw=2)
        ax.add_patch(circ)
        ax.text(xt,yt,r'$R={:.2f}$ mm'.format(r*scale),fontsize=16,color='w')

In [101]:
def extract_radius_of_curvature(image_path,radius_scale,plot=False):
    # Load image in grayscale, blur, and detect edges
    img = cv2.imread(image_path,0)
    blurred = cv2.GaussianBlur(img, (5,5), 0)
    edges = cv2.erode(cv2.dilate(cv2.Canny(blurred,30,100), None, iterations=1), None, iterations=1)
    
    # find contours in the edge map
    cnts = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    (cnts, _) = contours.sort_contours(cnts)
    
    # loop over the contours individually 
    # NOTE: the last circle in the loop is the dot scale
    radius_pixel = []
    x_text = []
    y_text = []
    x_center = []
    y_center = []
    for c in cnts:
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 100:
            continue
        # gets the x and y values and makes a list of them seperately
        x = np.array(c)[:,0,0]
        y = np.array(c)[:,0,1]
        # Get the center of each contour to use as the first guest to determine the center point of the circle
        M = cv2.moments(c)
        x_m = int(M["m10"] / M["m00"])
        x_text.append(x_m-100)
        y_m = int(M["m01"] / M["m00"])
        y_text.append(y_m)
        center_estimate = x_m, y_m
        
        # define a few functions for fitting
        def calc_R(xc, yc):
            #calculate the distance of each 2D points from the center (xc, yc)
            return np.sqrt((x-xc)**2 + (y-yc)**2)

        def f_2(c):
            #calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc)
            Ri = calc_R(*c)
            return Ri - Ri.mean()
        
        # use the x,y listed above and center_estimate to fit circle to arcs
        center_2, ier = optimize.leastsq(f_2, center_estimate)
        xc_2, yc_2 = center_2
        Ri_2 = calc_R(*center_2)
        # average all the radii to get mean radius for each ar
        R_2 = Ri_2.mean()
        radius_pixel.append(R_2)
        x_center.append(xc_2)
        y_center.append(yc_2)
        
    # convert to numpy array
    radius_pixel = np.array(radius_pixel)
    ##### Plot #####
    if plot:
        make_plot(img,edges,x_center,y_center,radius_pixel,x_text,y_text,radius_scale/R_2)
    
    # Normalize the radii by the dot size
    radius_physical = radius_pixel*radius_scale/R_2
        
    return img, x_center, y_center, radius_physical, radius_scale/R_2
    
    

In [103]:
img, xc, yc, radii, scale = extract_radius_of_curvature('data/small_r_05_340_2_E35_A5.jpg',3)

Data file structure

## day
### experiment
labeled by some number
* arrays
  * original image
  * fitted image
* metadata
  * path to image filename
  * cylinder radius
  * pitch
  * thickness
  * width
  * E value
  * A value
  * coil
  * physical scale (in mm/pixel)
* derived data
  * x location, y location
  * scale x,y
  * scale radii
  * radii (physical)
  * strain

## Writing Data

In [274]:
def commit_image(dbase_name,image_path,strain,radius_scale,metadata,date=None):
    # extract curvatures
    img,x_center,y_center,radius_physical,physical_scale = extract_radius_of_curvature(image_path,radius_scale)
    # separate LCE curves from scale curve
    x_center_scale = x_center.pop()
    y_center_scale = y_center.pop()
    radius_physical_scale = radius_physical[-1]
    radius_physical = radius_physical[:-1]
    # adjust metadata
    metadata['physical_conversion'] = physical_scale
    metadata['x_center_scale'] = x_center_scale
    metadata['y_center_scale'] = y_center_scale
    metadata['physical_radius_scale'] = radius_physical_scale
    # write to file
    if date is None:
        date = datetime.datetime.today()
    if not isinstance(date,datetime.datetime):
        date = datetime.datetime.strptime(date,'%m %d %Y')
    with h5py.File(dbase_name,'a') as hf:
        grp_name = date.strftime('%m_%d_%y')
        # create day group if it does not already exist
        if grp_name not in hf:
            grp = hf.create_group(grp_name)
        else:
            grp = hf[grp_name]
        # create image subgroup
        subgrp_num = len([k for k in grp.keys()])
        subgrp = grp.create_group('{}'.format(subgrp_num))
        # add datasets to subgroup
        dset = subgrp.create_dataset('image',data=img)
        dset.attrs['filepath'] = image_path
        subgrp.create_dataset('strain',data=strain)
        subgrp.create_dataset('physical_radii',data=radius_physical)
        subgrp.create_dataset('x_center',data=np.array(x_center))
        subgrp.create_dataset('y_center',data=np.array(y_center))
        # add metadata
        for key in metadata:
            subgrp.attrs[key] = metadata[key]

In [275]:
metadata = {
    'cylinder_radius':5,
    'pitch':0.1,
    'thickness':1,
    'width':7,
    'E_value':0.01,
    'A_value':9,
    'coil':0.5
}

In [276]:
commit_image('lce_curvature_dbase.h5','data/small_r_05_340_2_E35_A5.jpg',
             np.array([0,50,100]),3,metadata,date='10 31 2017')

## Accessing Data

In [287]:
class LCEImage(object):
    
    def __init__(self,dbase_path,top_level='/'):
        self.dbase_path = dbase_path
        self.top_level = top_level
        self.level_flag = len(list(filter(None,self.top_level.split('/'))))
        
    @property
    def meta(self):
        with h5py.File(self.dbase_path,'r') as hf:
            grp = hf[self.top_level]
            attrs = dict(grp.attrs)
        
        return attrs
    
    def peek(self,save2file=False):
        if 'image' not in self:
            return ValueError('Can only plot image datasets.')
        ### Plotting ###
        fig = plt.figure()
        ax = fig.gca()
        ax.imshow(self['image'],cmap = 'viridis')
        ax.set_xticks([])
        ax.set_yticks([])
        # plot curves
        for i,(x,y,r) in enumerate(zip(self['x_center'],self['y_center'],self['physical_radii'])):
            circ = Circle((x,y),r/self.meta['physical_conversion'],
                          facecolor='none',
                          edgecolor=sns.color_palette('deep')[i],
                          fill=False,
                          lw=2,
                          label=r'$R={:.2f}$ mm'.format(r))
            ax.add_patch(circ)
        # plot scale
        circ = Circle((self.meta['x_center_scale'],self.meta['y_center_scale']),
                      self.meta['physical_radius_scale']/self.meta['physical_conversion'],
                      facecolor='none',edgecolor='k',fill=False,lw=2,
                      label=r'$R={:.2f}$ mm'.format(self.meta['physical_radius_scale']))
        ax.add_patch(circ)
        ax.legend(loc='best',frameon=True,ncol=2)
        if save2file:
            fig.savefig(save2file,bbox_inches='tight')
    
    def to_dataframe(self):
        if 'image' not in self:
            return pandas.concat([k.to_dataframe() for k in self]).reset_index()
        df_dict = self.meta
        df_dict['date'] = datetime.datetime.strptime(list(filter(None,self.top_level.split('/')))[0],'%m_%d_%y')
        df_dict['physical_radii'] = self['physical_radii']
        df_dict['strain'] = self['strain']
        
        return pandas.DataFrame(df_dict)
    
    def __repr__(self):
        with h5py.File(self.dbase_path,'r') as hf:
            dset_names = [k for k in hf[self.top_level].keys()]
        dset_names = '\n'.join(dset_names)
        attr_names = '\n'.join(['{}'.format(k) for k in self.meta])
        return '''{}
Datasets
--------
{}

Attributes
----------
{}
'''.format(self.top_level,dset_names,attr_names)
        
    def __getitem__(self,key):
        if self.level_flag == 0:
            if type(key) is int:
                with h5py.File(self.dbase_path,'r') as hf:
                    dates = [k for k in hf.keys()]
                dates = sorted(dates,key=lambda x: datetime.datetime.strptime(x,'%m_%d_%y'))
                key = dates[key]
        elif self.level_flag == 1:
            key = str(key)
        with h5py.File(self.dbase_path,'r') as hf:
            grp = hf[self.top_level]
            if key not in grp:
                raise IndexError('{} not found in {}'.format(key,self.top_level))
            ds = grp[key]
            if isinstance(ds,h5py.Group):
                return LCEImage(self.dbase_path,top_level='/'.join([self.top_level,key]))
            else:
                return np.array(ds)
            
    def __contains__(self,key):
        with h5py.File(self.dbase_path,'r') as hf:
            flag = key in hf[self.top_level]
        return flag
            

In [288]:
lceimage = LCEImage('lce_curvature_dbase.h5')

In [291]:
lceimage.to_dataframe()

,level_0,index,A_value,E_value,coil,cylinder_radius,date,physical_conversion,physical_radii,physical_radius_scale,pitch,strain,thickness,width,x_center_scale,y_center_scale
0,0,0,9,0.01,0.5,5,2017-10-31,0.011598,1.133414,3.0,0.1,0,1,7,876.268487,373.997645
1,1,1,9,0.01,0.5,5,2017-10-31,0.011598,47.077844,3.0,0.1,50,1,7,876.268487,373.997645
2,2,2,9,0.01,0.5,5,2017-10-31,0.011598,1.197809,3.0,0.1,100,1,7,876.268487,373.997645
3,3,0,9,0.01,0.5,5,2017-10-31,0.011598,1.133414,3.0,0.1,0,1,7,876.268487,373.997645
4,4,1,9,0.01,0.5,5,2017-10-31,0.011598,47.077844,3.0,0.1,50,1,7,876.268487,373.997645
5,5,2,9,0.01,0.5,5,2017-10-31,0.011598,1.197809,3.0,0.1,100,1,7,876.268487,373.997645
6,6,0,9,0.01,0.5,5,2017-10-31,0.011598,1.133414,3.0,0.1,0,1,7,876.268487,373.997645
7,7,1,9,0.01,0.5,5,2017-10-31,0.011598,47.077844,3.0,0.1,50,1,7,876.268487,373.997645
8,8,2,9,0.01,0.5,5,2017-10-31,0.011598,1.197809,3.0,0.1,100,1,7,876.268487,373.997645
9,9,0,9,0.01,0.5,5,2017-10-31,0.011598,1.133414,3.0,0.1,0,1,7,876.268487,373.997645
